# Assignment 1

Code the algorithm of the Hadamard test that we covered in class using the Python SDK (below is a print screen of the Qmod for your convenience). Execute it from the SDK using 1,000 , 2,000 , 4,000 , 8,000 and 16,000 shots, and for each job calculate the (real part of the) expectation value using the formula $Re\{\bra{0}U_{QFT} \ket{0}\} = 2P_0 - 1$. Plot a graph of the expectation value as a function of the number of shots. Add the theoretical value to the graph. Explain the results. 

In [1]:
from classiq import *
from classiq.execution import ExecutionPreferences

In [2]:
# define main function

@qfunc
def main(
    expectation_value: Output[QBit],
    psi: Output[QArray],
    test: Output[QBit]
) -> None:
    
    # allocate the first qubit (acts as control for QFT)
    allocate(1, expectation_value)

    # define and allocate psi qubit for 4 qubit qft
    allocate(4, psi)

    # append H, CQFT, H to circuit
    swap_test(state1=expectation_value, state2=psi, test=test)

In [3]:
shots_list = [1000, 2000, 4000, 8000, 16000]
results = []

for shots in shots_list:
    
    model = create_model(main)

    # parametrize execution with 'shots' number of shots and then synthesize
    model = set_execution_preferences(model, execution_preferences=ExecutionPreferences(num_shots=shots))
    qprog = synthesize(model)

    # execute program
    res = execute(qprog).result()

    # calculate probability of zero from the counts obtained
    prob_0 = res[0].value.counts['0']/shots

    # Re(<psi | U_QFT | psi>) = 2*P(0) - 1
    re_expectation_value = 2*prob_0 - 1
    results.append(re_expectation_value)

KeyError: '0'

In [ ]:
results

QFT on one qubit is the same as Hadamard Transform  
We defined our $\psi$ ket as $|\psi> = 0$  
So, $<\psi|U_{QFT}|\psi> = <0|H|0> = 1/\sqrt{2}$

In [ ]:
# set theoretical value
theoretical = np.ones(5)/np.sqrt(2)

# plot the data
plt.plot(shots_list, results, color = 'red', label = 'experimental')
plt.plot(shots_list, theoretical, color = 'blue', label = 'theoretical')
plt.xlabel('Shots')
plt.ylabel('Expectation Value')
plt.legend()
plt.show()

We can see that the experimental value has some amount of noise in lower number of shots, and gets closer to the theoretical value as the number of shots increase